In [1]:
# Cargar el CSV
import pandas as pd
import numpy as np
from datetime import date, timedelta

consumo_granada = pd.read_csv('../data/processed/consumo_granada_cleaned.csv')

# Creamos una copia explícita para trabajar las features
df_features = consumo_granada.copy()

# Aseguramos formato fecha
df_features['timestamp'] = pd.to_datetime(df_features['timestamp'])

In [2]:
# 2. Extracción de componentes temporales
# Extraemos información básica que el modelo necesita entender
df_features['hour'] = df_features['timestamp'].dt.hour
df_features['month'] = df_features['timestamp'].dt.month
df_features['day_of_week'] = df_features['timestamp'].dt.dayofweek  # 0=Lunes, 6=Domingo
df_features['day_of_month'] = df_features['timestamp'].dt.day
df_features['year'] = df_features['timestamp'].dt.year  # Año para dashboard

print("Variables horarias básicas extraídas.")

Variables horarias básicas extraídas.


In [3]:
# 3. Transformación Cíclica del Tiempo (Seno y Coseno)
# Esto permite al modelo entender la continuidad entre las 23:00 y las 00:00

# Ciclo Diario (24 horas)
df_features['hour_sin'] = np.sin(2 * np.pi * df_features['hour'] / 24)
df_features['hour_cos'] = np.cos(2 * np.pi * df_features['hour'] / 24)

# Ciclo Anual (12 meses) - Importante para capturar estacionalidad
df_features['month_sin'] = np.sin(2 * np.pi * df_features['month'] / 12)
df_features['month_cos'] = np.cos(2 * np.pi * df_features['month'] / 12)

In [4]:
# Funcion para calcular las fechas de Semana Santa
def calcular_pascua(anio):
    """
    Calcula la fecha del Domingo de Resurrección para un año dado
    usando el algoritmo de Butcher (Calendario Gregoriano).
    """
    a = anio % 19
    b = anio // 100
    c = anio % 100
    d = b // 4
    e = b % 4
    f = (b + 8) // 25
    g = (b - f + 1) // 3
    h = (19 * a + b - d - g + 15) % 30
    i = c // 4
    k = c % 4
    l = (32 + 2 * e + 2 * i - h - k) % 7
    m = (a + 11 * h + 22 * l) // 451
    
    mes = (h + l - 7 * m + 114) // 31
    dia = ((h + l - 7 * m + 114) % 31) + 1
    
    return date(anio, mes, dia)

def obtener_dias_semana_santa(anio):
    domingo_resurreccion = calcular_pascua(anio)
    
    # Restamos días para obtener las fechas clave
    domingo_ramos = domingo_resurreccion - timedelta(days=7)
    jueves_santo = domingo_resurreccion - timedelta(days=3)
    viernes_santo = domingo_resurreccion - timedelta(days=2)
    
    return {
        "Domingo de Ramos": domingo_ramos,
        "Jueves Santo": jueves_santo,
        "Viernes Santo": viernes_santo,
        "Domingo de Resurrección": domingo_resurreccion
    }

In [5]:
# 4. Variables de Calendario (Sociales)
# A. Fin de Semana (Variable Crítica según EDA)
# Si es Sábado (5) o Domingo (6) -> 1, resto -> 0
df_features['is_weekend'] = df_features['day_of_week'].isin([5, 6]).astype(int)

# B. Festivos (Lógica completa para Granada)
# Fechas fijas nacionales/autonómicas clave
festivos_fijos = [
    (1, 1),   # Año Nuevo
    (6, 1),   # Reyes
    (28, 2),  # Día de Andalucía
    (1, 5),   # Día del Trabajo
    (15, 8),  # Asunción
    (12, 10), # Hispanidad
    (1, 11),  # Todos los Santos
    (6, 12),  # Constitución
    (8, 12),  # Inmaculada
    (25, 12)  # Navidad
]

# Pre-calcular todas las fechas de Semana Santa para los años del dataset
semana_santa_cache = {}
for year in df_features['timestamp'].dt.year.unique():
    fechas_ss = obtener_dias_semana_santa(year)
    # Guardamos Jueves Santo y Viernes Santo (festivos oficiales)
    semana_santa_cache[year] = [
        fechas_ss["Jueves Santo"],
        fechas_ss["Viernes Santo"]
    ]

# Función para detectar festivo (MEJORADA con Semana Santa)
def es_festivo(row):
    m, d = row.month, row.day
    year = row.year
    
    # 1. Chequear festivos fijos
    if (m, d) in festivos_fijos:
        return 1
    
    # 2. Chequear Semana Santa (Jueves y Viernes Santo)
    fecha_actual = date(year, m, d)
    if year in semana_santa_cache:
        if fecha_actual in semana_santa_cache[year]:
            return 1
    
    # 3. Día de la Cruz (3 de Mayo en Granada) - Muy importante localmente
    if m == 5 and d == 3:
        return 1
    
    # 4. Virgen de las Angustias (Último domingo septiembre - aprox 15-30 sept)
    # (Simplificación: marcamos el 15 de sep como patrona para el modelo)
    if m == 9 and d == 15:
        return 1
    
    return 0

# Aplicamos la función (tarda unos segundos)
df_features['is_holiday'] = df_features['timestamp'].apply(lambda x: es_festivo(x))

# Si es festivo entre semana, actúa como fin de semana
# Creamos una variable combinada "día_no_laborable"
df_features['is_non_working'] = ((df_features['is_weekend'] == 1) | (df_features['is_holiday'] == 1)).astype(int)

print("Calendario laboral generado (con Semana Santa incluida).")

Calendario laboral generado (con Semana Santa incluida).


In [6]:
# 5. Transformaciones Climáticas (Física)
# Elevamos al cuadrado para capturar la no-linealidad (Calefacción vs A/C)
df_features['temp_sq'] = df_features['temperature'] ** 2

print("Features climáticas creadas.")

Features climáticas creadas.


In [7]:
# 6. One-Hot Encoding de Zonas (Geografía)
# Usamos dtype='int8' para ahorrar memoria (ocupa 8 veces menos)

dummies_zona = pd.get_dummies(df_features['zone_name'], prefix='zona', dtype='int8')

# Unimos las nuevas columnas al dataset principal
df_features = pd.concat([df_features, dummies_zona], axis=1)

print(f"Zonas codificadas. Nuevas columnas añadidas: {dummies_zona.shape[1]}")

Zonas codificadas. Nuevas columnas añadidas: 20


In [8]:
# 7. Limpieza Final y Optimización de Decimales

# A. Borrado de columnas que sobran.
columnas_a_borrar = [
    'zone_id', 'zone_name', 'fecha', 'hora'
]
df_final = df_features.drop(columns=columnas_a_borrar, errors='ignore')

# B. ESTRATEGIA DE REDONDEO SELECTIVO
# Definimos las columnas "intocables" (Originales de sensor)
cols_protegidas = ['consumption_kwh', 'temperature']

# Obtenemos la lista de columnas que SÍ queremos redondear
# (Son todas las columnas del DF menos las protegidas)
cols_a_redondear = [col for col in df_final.columns if col not in cols_protegidas]

# Aplicamos el redondeo SOLO a esas columnas
df_final[cols_a_redondear] = df_final[cols_a_redondear].round(4)

# Muestra para verificar:
df_final.head()

,timestamp,consumption_kwh,temperature,hour,month,day_of_month,day_of_week,year,hour_sin,hour_cos,...,zona_Mercagranada,zona_Norte_Almanjayar,zona_Pedro_Antonio,zona_Periodistas,zona_Plaza_Toros,zona_Pts_Tecnologico,zona_Realejo,zona_Sacromonte,zona_Zaidin_Nuevo,zona_Zaidin_Vergeles
0,2015-01-01,359.454,-0.09,0,1,1,3,2015,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,2015-01-01,439.901,-0.52,0,1,1,3,2015,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,2015-01-01,196.527,-0.13,0,1,1,3,2015,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,2015-01-01,436.102,-0.33,0,1,1,3,2015,0.0,1.0,...,0,0,0,0,0,0,1,0,0,0
4,2015-01-01,213.958,-0.39,0,1,1,3,2015,0.0,1.0,...,0,0,0,0,0,0,0,1,0,0


In [9]:
# Guardar dataset listo para entrenar
df_final.to_csv('../data/processed/consumo_granada_modelo.csv', index=False)